In [ ]:
pip install transformers

In [ ]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ECE1786/Project/Textfiles

/content/drive/MyDrive/ECE1786/Project/Textfiles


In [ ]:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files='/content/drive/MyDrive/ECE1786/Project/Textfiles/master', vocab_size=50000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model("/content/drive/MyDrive/ECE1786/Project/Tokenizer Files", "ECE1786Tokenizer0")

['/content/drive/MyDrive/ECE1786/Project/Tokenizer Files/ECE1786Tokenizer0-vocab.json',
 '/content/drive/MyDrive/ECE1786/Project/Tokenizer Files/ECE1786Tokenizer0-merges.txt']

In [ ]:
#example tokenization
tokenizer = ByteLevelBPETokenizer(
    "/content/drive/MyDrive/ECE1786/Project/Tokenizer Files/ECE1786Tokenizer0-vocab.json",
    "/content/drive/MyDrive/ECE1786/Project/Tokenizer Files/ECE1786Tokenizer0-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

print(
    tokenizer.encode("CIA").ids
)

[0, 2195, 2]


In [ ]:
#take training lines file and generate masked training examples which can be passed through the tokenizer into a model
import csv
sep = " " #separator for joining training sentences
with open('/content/drive/MyDrive/ECE1786/Project/CSVs/traininglines.csv', 'r') as tl:
  csvreader = csv.reader(tl)
  with open('/content/drive/MyDrive/ECE1786/Project/CSVs/trainingdata.csv','w') as td:
    csvwriter = csv.writer(td)
    for row in csvreader:
      rowcopy = row.copy()
      splitsentence = rowcopy[0].split()
      target = rowcopy[1]
      for i, word in enumerate(splitsentence):
        if word == target:
          splitsentence[i] = "<mask>" #start simple: only try to guess keywords. will expand to guessing multiple words and nearby words
      maskedsentence = sep.join(splitsentence)
      csvwriter.writerow([maskedsentence] + [target])